In [2]:
import numpy as np
import pandas as pd
import re

from utils.data import get_unique_types_of_col, get_null_counts, get_all_unique_types

In [3]:
df = pd.read_csv('../data/dayobs.csv')
df.columns

Index(['Date', 'Region', 'Sunspot Number', 'Size', 'Magnetic Classification',
       'Sunspot Classification', 'Location'],
      dtype='object')

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Zurich Class'] = df['Sunspot Classification'].str[0]
df['Penumbra Class'] = df['Sunspot Classification'].str[1]
df['Compactness Class'] = df['Sunspot Classification'].str[2]
df.drop(['Sunspot Classification', 'Magnetic Classification', 'Date'], axis=1, inplace=True)
df

,Region,Sunspot Number,Size,Location,Year,Month,Day,Zurich Class,Penumbra Class,Compactness Class
0,7968,3,0,N02E41,1996,6,4,A,X,X
1,7968,2,10,N02E28,1996,6,5,B,X,O
2,7969,1,0,N09W33,1996,6,5,A,X,X
3,7968,4,20,N03E12,1996,6,6,C,R,O
4,7969,4,10,N08W44,1996,6,6,B,X,O
...,...,...,...,...,...,...,...,...,...,...
39856,13731,1,10,S16E21,2024,6,30,A,X,X
39857,13732,4,30,S18W14,2024,6,30,C,A,O
39858,13733,4,40,N05E13,2024,6,30,D,A,O
39859,13734,4,60,N07E53,2024,6,30,D,A,O


In [5]:
df['Zurich Class'].unique()

array(['A', 'B', 'C', 'D', 'H', 'E', 'F'], dtype=object)

In [6]:
df['Penumbra Class'].unique()

array(['X', 'R', 'S', 'A', 'H', 'K'], dtype=object)

In [7]:
df['Compactness Class'].unique()

array(['X', 'O', 'I', 'C'], dtype=object)

## NOAA dataset

In [8]:
sunspot_cols_and_widths = {
  "Data Code": 2,
  "Year": 2,
  "Month": 2,
  "Day": 2,
  "blank1": 1,
  "Universal Time of Observation": 4,
  "blank2": 1,
  "Location": 6,
  "Mt Wilson Magnetic Classification": 4,
  "blank3": 1,
  "Max Magnetic Field Strength": 1,
  "blank4": 1,
  "Mount Wilson Spot Group Number": 5,
  "Subscript": 1,
  "NOAA Sunspot Group Number": 5, # NOAA/USAF sunspot group number
  "Subscript NOAA": 1,  # This field is used by WDC  to subscript the NOAA/USAF sunspot number, for example 4013, 4013A.
  "Zurich Class": 1,  # USAF only
  "Penumbra Class": 1,  # USAF only
  "Compactness Class": 1, # USAF only
  "blank5": 1,
  "Number of Spots": 2, # USAF only
  "blank6": 1,
  "Longitudinal Extent": 2, # In degrees | USAF only
  "Area in millionths of solar hemisphere": 4, # USAF only
  "blank7": 1,
  "Individual CMP Year": 2,
  "Individual CMP Month": 2,
  "Individual CMP Day": 4,
  "blank8": 1,
  "Regional CMP Year": 2,
  "Regional CMP Month": 2,
  "Regional CMP Day": 4,
  "blank9": 1,
  "Station Serial Number": 3,
  "blank10": 1,
  "Quality": 1, # 1=very poor, 2=poor, 3=fair, 4=good, 5=excellent
  "Station": 4, # 4-letter station abbreviation
}

In [9]:
df_old_data = pd.read_fwf("../spots1981-2017.txt", header=None,
                 widths=list(sunspot_cols_and_widths.values()),
                 names=list(sunspot_cols_and_widths.keys()))

In [10]:
df_old_data.drop(columns=df_old_data.filter(like='blank').columns, inplace=True)
df_old_data.drop(['Data Code', 'Max Magnetic Field Strength', 'Subscript',
                  'Individual CMP Year', 'Individual CMP Month', 'Individual CMP Day',
                  'Regional CMP Year', 'Regional CMP Month', 'Regional CMP Day',
                  'Station Serial Number', 'Station', 'Universal Time of Observation',
                  'Mount Wilson Spot Group Number', 'Subscript NOAA',
                  'Mt Wilson Magnetic Classification',
                  'Quality', 'Longitudinal Extent'], axis=1, inplace=True)
df_old_data

,Year,Month,Day,Location,NOAA Sunspot Group Number,Zurich Class,Penumbra Class,Compactness Class,Number of Spots,Area in millionths of solar hemisphere
0,81,12,1,N13W46,3478,E,K,I,32.0,600.0
1,81,12,1,N13W46,3478,E,H,I,16.0,530.0
2,81,12,1,N12W47,3478,E,H,O,22.0,710.0
3,81,12,1,N15W56,3478,E,H,I,31.0,670.0
4,81,12,1,N13W55,3478,E,H,I,17.0,430.0
...,...,...,...,...,...,...,...,...,...,...
271878,17,6,29,N18W35,12664,C,S,O,2.0,40.0
271879,17,6,29,N19W42,12664,C,S,O,3.0,60.0
271880,17,6,30,N15W45,12664,C,S,O,2.0,60.0
271881,17,6,30,N18W47,12664,H,S,X,1.0,30.0


In [11]:
get_null_counts(df_old_data)

NOAA Sunspot Group Number                    37
Zurich Class                              36960
Penumbra Class                            37027
Compactness Class                         76113
Number of Spots                           36845
Area in millionths of solar hemisphere    50532
dtype: int64

In [12]:
df_old_data['Year'] = df_old_data['Year'].astype(int).apply(lambda x: 1900 + x if x > 50 else 2000 + x)
df_old_data['Month'] = df_old_data['Month'].astype(int)
df_old_data['Day'] = df_old_data['Day'].astype(int)

df_old_data

,Year,Month,Day,Location,NOAA Sunspot Group Number,Zurich Class,Penumbra Class,Compactness Class,Number of Spots,Area in millionths of solar hemisphere
0,1981,12,1,N13W46,3478,E,K,I,32.0,600.0
1,1981,12,1,N13W46,3478,E,H,I,16.0,530.0
2,1981,12,1,N12W47,3478,E,H,O,22.0,710.0
3,1981,12,1,N15W56,3478,E,H,I,31.0,670.0
4,1981,12,1,N13W55,3478,E,H,I,17.0,430.0
...,...,...,...,...,...,...,...,...,...,...
271878,2017,6,29,N18W35,12664,C,S,O,2.0,40.0
271879,2017,6,29,N19W42,12664,C,S,O,3.0,60.0
271880,2017,6,30,N15W45,12664,C,S,O,2.0,60.0
271881,2017,6,30,N18W47,12664,H,S,X,1.0,30.0


In [13]:
df_old_data['Zurich Class'].unique()

array(['E', nan, 'C', 'B', 'A', 'D', 'H', 'F', 'X', '?'], dtype=object)

In [14]:
df_old_data['Penumbra Class'].unique()

array(['K', 'H', 'S', nan, 'A', 'X', 'R', 'O'], dtype=object)

In [15]:
df_old_data['Compactness Class'].unique()

array(['I', 'O', nan, 'X', 'C', '?', '0'], dtype=object)

In [16]:
def mcintosh_class_fix(input_string):
  if input_string == '0':
    return 'O'
  elif input_string == '?':
    return np.nan
  else:
    return input_string

In [17]:
df_old_data['Zurich Class'] = df_old_data['Zurich Class'].apply(mcintosh_class_fix)
df_old_data['Penumbra Class'] = df_old_data['Penumbra Class'].apply(mcintosh_class_fix)
df_old_data['Compactness Class'] = df_old_data['Compactness Class'].apply(mcintosh_class_fix)
df_old_data['Compactness Class'].unique()

array(['I', 'O', nan, 'X', 'C'], dtype=object)

In [18]:
df.columns

Index(['Region', 'Sunspot Number', 'Size', 'Location', 'Year', 'Month', 'Day',
       'Zurich Class', 'Penumbra Class', 'Compactness Class'],
      dtype='object')

In [19]:
df_old_data.columns

Index(['Year', 'Month', 'Day', 'Location', 'NOAA Sunspot Group Number',
       'Zurich Class', 'Penumbra Class', 'Compactness Class',
       'Number of Spots', 'Area in millionths of solar hemisphere'],
      dtype='object')

### Items need to be cross-checked:
| New Data | Old Data                               |
|----------|----------------------------------------|
| Region   | NOAA Sunspot Group Number              |
| Size     | Area in millionths of solar hemisphere |

### Columns name update:
| New Data                    | Old Data                               |
|-----------------------------|----------------------------------------|
| [x] Region                  | NOAA Sunspot Group Number              |
| [x] Size                    | Area in millionths of solar hemisphere |
| [x] Sunspot Number          | Number of Spots                        |
| [-] Magnetic Classification | Mt Wilson Magnetic Classification      |

In [20]:
df_old_data.rename(columns={
  'NOAA Sunspot Group Number': 'Region',
  'Area in millionths of solar hemisphere': 'Size',
  'Number of Spots': 'Sunspot Number',
  # 'Mt Wilson Magnetic Classification': 'Magnetic Classification',
}, inplace=True)
df_old_data

,Year,Month,Day,Location,Region,Zurich Class,Penumbra Class,Compactness Class,Sunspot Number,Size
0,1981,12,1,N13W46,3478,E,K,I,32.0,600.0
1,1981,12,1,N13W46,3478,E,H,I,16.0,530.0
2,1981,12,1,N12W47,3478,E,H,O,22.0,710.0
3,1981,12,1,N15W56,3478,E,H,I,31.0,670.0
4,1981,12,1,N13W55,3478,E,H,I,17.0,430.0
...,...,...,...,...,...,...,...,...,...,...
271878,2017,6,29,N18W35,12664,C,S,O,2.0,40.0
271879,2017,6,29,N19W42,12664,C,S,O,3.0,60.0
271880,2017,6,30,N15W45,12664,C,S,O,2.0,60.0
271881,2017,6,30,N18W47,12664,H,S,X,1.0,30.0


In [21]:
df_old_data.shape

(271883, 10)

In [22]:
# df_old_data.dropna(subset=['Sunspot Number'], inplace=True)
# df_old_data.shape

# Cross checking

In [23]:
df_old_data = df_old_data[df.columns]

In [24]:
df['Year'].describe()[['min', 'max']]

min    1996.0
max    2024.0
Name: Year, dtype: float64

In [25]:
df_old_data['Year'].describe()[['min', 'max']]

min    1981.0
max    2017.0
Name: Year, dtype: float64

In [26]:
df_old_data[df_old_data['Year'] == 2017]['Month'].describe()[['min', 'max']]

min    1.0
max    6.0
Name: Month, dtype: float64

In [27]:
df_old_data[(df_old_data['Year'] == 2017) & (df_old_data['Month'] == 6)]['Day'].describe()[['min', 'max']]

min     1.0
max    30.0
Name: Day, dtype: float64

# Latest entry of old data is 30/06/2017, Consider taking new data after that

In [28]:
# take all from year 2017 to 2023
new_data_after_2017_06 = df[(df['Year'] >= 2017) & (df['Year'] <= 2023)]

# Remove all before July 2017
new_data_after_2017_06 = new_data_after_2017_06[~((new_data_after_2017_06['Year'] == 2017) & (new_data_after_2017_06['Month'] < 7)) ]
new_data_after_2017_06

,Region,Sunspot Number,Size,Location,Year,Month,Day,Zurich Class,Penumbra Class,Compactness Class
31731,12664,1,50,N17W58,2017,7,1,H,S,X
31732,12664,1,20,N18W71,2017,7,2,H,S,X
31733,12664,1,20,N18W85,2017,7,3,H,S,X
31734,12665,1,70,S05E78,2017,7,6,H,S,X
31735,12665,6,140,S06E64,2017,7,7,D,A,I
...,...,...,...,...,...,...,...,...,...,...
38244,13534,12,150,S13E16,2023,12,30,D,A,O
38245,13531,1,160,S19W77,2023,12,31,H,S,X
38246,13533,2,20,N15W78,2023,12,31,B,X,O
38247,13534,10,120,S12E03,2023,12,31,D,A,O


In [29]:
df_merged = pd.concat([df_old_data, new_data_after_2017_06], ignore_index=True)
df_merged

,Region,Sunspot Number,Size,Location,Year,Month,Day,Zurich Class,Penumbra Class,Compactness Class
0,3478,32.0,600.0,N13W46,1981,12,1,E,K,I
1,3478,16.0,530.0,N13W46,1981,12,1,E,H,I
2,3478,22.0,710.0,N12W47,1981,12,1,E,H,O
3,3478,31.0,670.0,N15W56,1981,12,1,E,H,I
4,3478,17.0,430.0,N13W55,1981,12,1,E,H,I
...,...,...,...,...,...,...,...,...,...,...
278396,13534,12.0,150.0,S13E16,2023,12,30,D,A,O
278397,13531,1.0,160.0,S19W77,2023,12,31,H,S,X
278398,13533,2.0,20.0,N15W78,2023,12,31,B,X,O
278399,13534,10.0,120.0,S12E03,2023,12,31,D,A,O


# keep float in Sunspot number and Size also keep Nan

In [30]:
get_unique_types_of_col(df_merged['Region'])

{float, int, str}

In [31]:
df_merged['Region'].astype(str).unique()

array(['3478', '3480', '3484', ..., '13534', '13535', '13536'],
      dtype=object)

In [32]:
df_merged['Region'] = df_merged['Region'].astype(str).replace('////', np.nan)

In [33]:
get_null_counts(df_merged)

Region                2112
Sunspot Number       36845
Size                 50532
Zurich Class         36965
Penumbra Class       37027
Compactness Class    76114
dtype: int64

In [34]:
get_all_unique_types(df_merged)

{'Region': {float, str},
 'Sunspot Number': {float},
 'Size': {float},
 'Location': {str},
 'Year': {int},
 'Month': {int},
 'Day': {int},
 'Zurich Class': {float, str},
 'Penumbra Class': {float, str},
 'Compactness Class': {float, str}}

# Remove 1981, it has only December data

In [39]:
df_merged = df_merged[df_merged['Year'] != 1981]

In [40]:
df_merged.to_csv('../data/sunspot-processed-2.csv', index=False)